In [1]:
import numpy as np
import scipy.special
import matplotlib.pyplot as plt
import pandas as pd

### Create rnorm_pre() function

In [17]:
# ref: https://github.com/debruine/faux/blob/master/R/rnorm_pre.R
import numpy as np
import statsmodels.api as sm

def sample_from_pop(n=100, mu=0, sd=1, r=0):
    r_sd = np.sqrt(1/n) * np.sqrt(1 - r**2)
    sample_r = np.random.normal(r, r_sd)
    sample_r = np.clip(sample_r, -1, 1)
    
    mu_sd = sd / np.sqrt(n)
    sample_mu = np.random.normal(mu, mu_sd)
    
    sd_sd = sd / np.sqrt(2 * n)
    sample_sd = np.random.normal(sd, sd_sd)
    
    return {'mu': sample_mu, 'sd': sample_sd, 'r': sample_r}

def rnorm_pre(x, mu=0, sd=1, r=0, empirical=False, threshold=1e-12):
    if isinstance(x, np.ndarray) and len(x.shape) == 1:
        x = x.reshape(-1, 1)
    elif isinstance(x, np.ndarray) and len(x.shape) == 2:
        pass
    else:
        raise ValueError("x must be a vector or a 2D array")

    n, d = x.shape
    rho = np.repeat(r, d)

    if not empirical:
        sample_params = sample_from_pop(n, mu, sd, rho)
        mu = sample_params['mu']
        sd = sample_params['sd']
        rho = sample_params['r']

    x = (x - np.mean(x, axis=0)) / np.std(x, axis=0)
    y = np.random.normal(size=n)
        
    e = sm.OLS(y, x).fit().resid

    if d == 1:
        z = rho * x[:, 0] + np.sqrt(1 - rho**2) * e
    else:
        u, s, vh = svd(x, full_matrices=False)
        x_dual = np.dot(u * (n - 1), vh)
        sigma2 = ((1 - np.dot(np.dot(rho, x_dual.T), rho)) / np.var(e)).clip(min=0)

        if sigma2 >= 0:
            sigma = np.sqrt(sigma2)
            z = np.dot(x_dual, rho) + sigma * e
        else:
            print("Correlations are impossible.")
            z = np.zeros(n)

    return mu + sd * z


In [18]:
# test the function
np.random.seed(12)
a = np.random.normal(0, 1, size=n_samples)
b = rnorm_pre(a, mu=0, sd=1, r=0.78)
print(round(np.corrcoef(a,b)[0,1], 3))

0.786


### Setting

In [2]:
n_samples = 2000 # number of observations
n_features = 10  # number of covariates

np.random.seed(1)
W = np.random.binomial(1, scipy.special.expit(np.random.normal(0, 1, size=n_samples)))

print('W.shape',W.shape)

pd.DataFrame(W).to_csv('./data_for_R/Low-dim-NEW/W.csv', index=False)

W.shape (2000,)


In [3]:
np.random.seed(12)
e = np.random.normal(0, 1, size=n_samples) # gaussian noise 1, used in Y

print('e.shape',e.shape)

e.shape (2000,)


In [4]:
np.random.seed(123)
h = np.random.normal(0, 1, size=n_samples) # gaussian noise 2, used in mu

print('h.shape',h.shape)

h.shape (2000,)


### no TE & no HTE

In [6]:
np.random.seed(1234)
X = np.random.normal(0, 1, size=(n_samples, n_features))

np.random.seed(12345)
mu = 2*h
Y = mu + e

print('mu.shape',mu.shape)
print('Y.shape',Y.shape)
print('cor(Y,W)',round(np.corrcoef(Y,W)[0,1], 3))
for col in range(X.shape[1]):
    print('cor(mu,X)','col',col,round(np.corrcoef(mu,X[:,col])[0,1], 3))
for col in range(X.shape[1]):
    print('cor(Y,X) ','col',col,round(np.corrcoef(Y,X[:,col])[0,1], 3))

pd.DataFrame(X).to_csv('./data_for_R/Low-dim-NEW/X_noHTE.csv', index=False)    
pd.DataFrame(Y).to_csv('./data_for_R/Low-dim-NEW/Y_noTE_noHTE.csv', index=False)

mu.shape (2000,)
Y.shape (2000,)
cor(Y,W) -0.003
cor(mu,X) col 0 -0.02
cor(mu,X) col 1 -0.001
cor(mu,X) col 2 -0.016
cor(mu,X) col 3 -0.036
cor(mu,X) col 4 0.004
cor(mu,X) col 5 -0.027
cor(mu,X) col 6 0.071
cor(mu,X) col 7 -0.022
cor(mu,X) col 8 -0.004
cor(mu,X) col 9 0.005
cor(Y,X)  col 0 -0.015
cor(Y,X)  col 1 0.01
cor(Y,X)  col 2 -0.029
cor(Y,X)  col 3 -0.012
cor(Y,X)  col 4 0.0
cor(Y,X)  col 5 -0.018
cor(Y,X)  col 6 0.058
cor(Y,X)  col 7 0.0
cor(Y,X)  col 8 -0.018
cor(Y,X)  col 9 0.007


### TE & no HTE

In [7]:
np.random.seed(1234)
X = np.random.normal(0, 1, size=(n_samples, n_features)) # same as above

np.random.seed(12345)
mu = W + 2*h 
mu_W = np.matmul(mu.reshape(n_samples,-1), W.reshape(-1, n_samples))
Y = mu_W.mean(axis=1) + e

print('cor(mu,,W)',round(np.corrcoef(mu,W)[0,1], 3))
print('cor(Y,W)',round(np.corrcoef(Y,W)[0,1], 3))
for col in range(X.shape[1]):
    print('cor(mu,X)','col',col,round(np.corrcoef(mu,X[:,col])[0,1], 3))
for col in range(X.shape[1]):
    print('cor(Y,X) ','col',col,round(np.corrcoef(Y,X[:,col])[0,1], 3))
    
pd.DataFrame(Y).to_csv('./data_for_R/Low-dim-NEW/Y_TE_noHTE.csv', index=False)

cor(mu,,W) 0.248
cor(Y,W) 0.173
cor(mu,X) col 0 -0.013
cor(mu,X) col 1 0.004
cor(mu,X) col 2 -0.02
cor(mu,X) col 3 -0.03
cor(mu,X) col 4 0.002
cor(mu,X) col 5 -0.022
cor(mu,X) col 6 0.053
cor(mu,X) col 7 -0.022
cor(mu,X) col 8 -0.014
cor(mu,X) col 9 -0.008
cor(Y,X)  col 0 -0.005
cor(Y,X)  col 1 0.02
cor(Y,X)  col 2 -0.038
cor(Y,X)  col 3 0.009
cor(Y,X)  col 4 -0.004
cor(Y,X)  col 5 -0.007
cor(Y,X)  col 6 0.031
cor(Y,X)  col 7 0.014
cor(Y,X)  col 8 -0.032
cor(Y,X)  col 9 -0.002


### TE & HTE (only protective factors)

In [22]:
np.random.seed(1234)
X = np.random.normal(0, 1, size=(n_samples, n_features)) 

np.random.seed(12345)
mu = W + 2*h 

np.random.seed(123456)
X[:,0] = X[:,0] + rnorm_pre(mu, mu=0, sd=1, r=0.5)

np.random.seed(123456)
X[:,1] = X[:,1] + rnorm_pre(mu, mu=0, sd=1, r=0.5)

g = X.mean(axis=1)
mu_tilde = 2*mu + g
Y = mu_tilde*W + e

print('cor(mu_tilde,W)',round(np.corrcoef(mu,W)[0,1], 3))
print('cor(Y,W)',round(np.corrcoef(Y,W)[0,1], 3))
for col in range(X.shape[1]):
    print('cor(mu_tilde,X)','col',col,round(np.corrcoef(mu,X[:,col])[0,1], 3))
for col in range(X.shape[1]):
    print('cor(Y,X) ','col',col,round(np.corrcoef(Y,X[:,col])[0,1], 3))

pd.DataFrame(X).to_csv('./data_for_R/Low-dim-NEW/X_TE_protHTE.csv', index=False)
pd.DataFrame(Y).to_csv('./data_for_R/Low-dim-NEW/Y_TE_protHTE.csv', index=False)

cor(mu_tilde,W) 0.248
cor(Y,W) 0.301
cor(mu_tilde,X) col 0 0.351
cor(mu_tilde,X) col 1 0.36
cor(mu_tilde,X) col 2 -0.02
cor(mu_tilde,X) col 3 -0.03
cor(mu_tilde,X) col 4 0.002
cor(mu_tilde,X) col 5 -0.022
cor(mu_tilde,X) col 6 0.053
cor(mu_tilde,X) col 7 -0.022
cor(mu_tilde,X) col 8 -0.014
cor(mu_tilde,X) col 9 -0.008
cor(Y,X)  col 0 0.275
cor(Y,X)  col 1 0.287
cor(Y,X)  col 2 -0.021
cor(Y,X)  col 3 0.043
cor(Y,X)  col 4 0.011
cor(Y,X)  col 5 0.027
cor(Y,X)  col 6 0.051
cor(Y,X)  col 7 0.026
cor(Y,X)  col 8 -0.023
cor(Y,X)  col 9 0.012


### TE & HTE (only risk factors)

In [23]:
np.random.seed(1234)
X = np.random.normal(0, 1, size=(n_samples, n_features)) 

np.random.seed(12345)
mu = W + 2*h 

np.random.seed(123456)
X[:,2] = X[:,2] - rnorm_pre(mu, mu=0, sd=1, r=0.5)

np.random.seed(123456)
X[:,3] = X[:,3] - rnorm_pre(mu, mu=0, sd=1, r=0.5)

g = X.mean(axis=1)
mu_tilde = 2*mu + g
Y = mu_tilde*W + e

print('cor(mu_tilde,W)',round(np.corrcoef(mu,W)[0,1], 3))
print('cor(Y,W)',round(np.corrcoef(Y,W)[0,1], 3))
for col in range(X.shape[1]):
    print('cor(mu_tilde,X)','col',col,round(np.corrcoef(mu,X[:,col])[0,1], 3))
for col in range(X.shape[1]):
    print('cor(Y,X) ','col',col,round(np.corrcoef(Y,X[:,col])[0,1], 3))
    
pd.DataFrame(X).to_csv('./data_for_R/Low-dim-NEW/X_TE_riskHTE.csv', index=False)    
pd.DataFrame(Y).to_csv('./data_for_R/Low-dim-NEW/Y_TE_riskHTE.csv', index=False)

cor(mu_tilde,W) 0.248
cor(Y,W) 0.309
cor(mu_tilde,X) col 0 -0.013
cor(mu_tilde,X) col 1 0.004
cor(mu_tilde,X) col 2 -0.376
cor(mu_tilde,X) col 3 -0.364
cor(mu_tilde,X) col 4 0.002
cor(mu_tilde,X) col 5 -0.022
cor(mu_tilde,X) col 6 0.053
cor(mu_tilde,X) col 7 -0.022
cor(mu_tilde,X) col 8 -0.014
cor(mu_tilde,X) col 9 -0.008
cor(Y,X)  col 0 -0.002
cor(Y,X)  col 1 0.018
cor(Y,X)  col 2 -0.257
cor(Y,X)  col 3 -0.197
cor(Y,X)  col 4 0.013
cor(Y,X)  col 5 0.027
cor(Y,X)  col 6 0.051
cor(Y,X)  col 7 0.029
cor(Y,X)  col 8 -0.02
cor(Y,X)  col 9 0.015


### TE & HTE (both protective & risk factors)

In [24]:
np.random.seed(1234)
X = np.random.normal(0, 1, size=(n_samples, n_features)) 

np.random.seed(12345)
mu = W + 2*h 

np.random.seed(123456)
X[:,0] = X[:,0] + rnorm_pre(mu, mu=0, sd=1, r=0.5)

np.random.seed(1234567)
X[:,1] = X[:,1] + rnorm_pre(mu, mu=0, sd=1, r=0.5)

np.random.seed(12345678)
X[:,2] = X[:,2] - rnorm_pre(mu, mu=0, sd=1, r=0.5)

np.random.seed(123456789)
X[:,3] = X[:,3] - rnorm_pre(mu, mu=0, sd=1, r=0.5)

g = X.mean(axis=1)
mu_tilde = 2*mu + g
Y = mu_tilde*W + e

print('cor(mu_tilde,W)',round(np.corrcoef(mu,W)[0,1], 3))
print('cor(Y,W)',round(np.corrcoef(Y,W)[0,1], 3))
for col in range(X.shape[1]):
    print('cor(mu_tilde,X)','col',col,round(np.corrcoef(mu,X[:,col])[0,1], 3))
for col in range(X.shape[1]):
    print('cor(Y,X) ','col',col,round(np.corrcoef(Y,X[:,col])[0,1], 3))

pd.DataFrame(X).to_csv('./data_for_R/Low-dim-NEW/X_TE_bothHTE.csv', index=False)    
pd.DataFrame(Y).to_csv('./data_for_R/Low-dim-NEW/Y_TE_bothHTE.csv', index=False)

cor(mu_tilde,W) 0.248
cor(Y,W) 0.306
cor(mu_tilde,X) col 0 0.351
cor(mu_tilde,X) col 1 0.359
cor(mu_tilde,X) col 2 -0.375
cor(mu_tilde,X) col 3 -0.409
cor(mu_tilde,X) col 4 0.002
cor(mu_tilde,X) col 5 -0.022
cor(mu_tilde,X) col 6 0.053
cor(mu_tilde,X) col 7 -0.022
cor(mu_tilde,X) col 8 -0.014
cor(mu_tilde,X) col 9 -0.008
cor(Y,X)  col 0 0.267
cor(Y,X)  col 1 0.25
cor(Y,X)  col 2 -0.259
cor(Y,X)  col 3 -0.239
cor(Y,X)  col 4 0.013
cor(Y,X)  col 5 0.026
cor(Y,X)  col 6 0.05
cor(Y,X)  col 7 0.028
cor(Y,X)  col 8 -0.022
cor(Y,X)  col 9 0.013
